# Twitter data

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
twitter_df = pd.read_csv('/Users/menglinzhou/Library/CloudStorage/OneDrive-Personal/Projects/microsoft-hackathon-24/ML-LoRA-E5/twitter_data/twitter_processed.csv')

In [4]:
def sample_with_seed(group, seed=42, size=100):  # Introduce seed parameter
    np.random.seed(seed)  # Set seed before sampling
    return group.sample(min(len(group), size))

In [5]:
twitter_sample_df = twitter_df.groupby(['labels'], group_keys=False).apply(lambda x: sample_with_seed(x, seed=100, size=10000))

/var/folders/mh/mgs_z6397_x1r2rf93xvbk3h0000gn/T/ipykernel_26852/1742114485.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  twitter_sample_df = twitter_df.groupby(['labels'], group_keys=False).apply(lambda x: sample_with_seed(x, seed=100, size=10000))


In [6]:
twitter_sample_df['labels'].value_counts()

labels
0    10000
1    10000
Name: count, dtype: int64

In [7]:
missing_values = twitter_sample_df.isnull().sum()
print(missing_values)

text      0
labels    0
dtype: int64


In [8]:
twitter_sample_df = pd.DataFrame(
    {'text': twitter_sample_df['text'],
     'domain': 'twitter',
     'model': 'gpt4o-mini',
     'attack': 'none',
     'labels': twitter_sample_df['labels']}
)

In [9]:
twitter_sample_df['labels'].value_counts()

labels
0    10000
1    10000
Name: count, dtype: int64

# Raid data

In [10]:
raid_ai_df = pd.read_csv('raid_ai.csv')

In [11]:
raid_ai_df = raid_ai_df.groupby(['domain', 'model'], group_keys=False).apply(lambda x: sample_with_seed(x, seed=100, size = 2000))

/var/folders/mh/mgs_z6397_x1r2rf93xvbk3h0000gn/T/ipykernel_26852/3600643269.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  raid_ai_df = raid_ai_df.groupby(['domain', 'model'], group_keys=False).apply(lambda x: sample_with_seed(x, seed=100, size = 2000))


In [12]:
raid_human_df = pd.read_csv('raid_human.csv')

In [14]:
raid_ai_sub = pd.DataFrame(
    {'text': raid_ai_df['generation'],
     'domain': raid_ai_df['domain'],
     'model': raid_ai_df['model'],
     'attack': raid_ai_df['attack'],
     'labels': 1}
)

raid_human_sub = pd.DataFrame(
    {'text': raid_human_df['generation'],
     'domain': raid_human_df['domain'],
     'model': raid_human_df['model'],
     'attack': raid_human_df['attack'],
     'labels': 0}
)

In [16]:
raid_ai_sub['labels'].value_counts()

labels
1    128000
Name: count, dtype: int64

In [17]:
raid_human_sub['labels'].value_counts()

labels
0    80000
Name: count, dtype: int64

In [25]:
raid_train_df = pd.concat([raid_human_sub, raid_ai_sub], axis = 0)

In [26]:
raid_train_df['labels'].value_counts()

labels
1    128000
0     80000
Name: count, dtype: int64

In [27]:
df = pd.concat([raid_train_df, twitter_sample_df], axis=0)

In [28]:
df['labels'].value_counts()

labels
1    138000
0     90000
Name: count, dtype: int64

In [29]:
df = df.sample(frac=1).reset_index(drop=True)

In [64]:
display(df)

,text,domain,model,attack,labels
0,\n\nKatraj is a hill station in the Indian sta...,wiki,gpt3,alternative_spelling,1
1,Underrated doesn't deserve hate. Back to Futur...,reviews,human,article_deletion,0
2,"In 1875, Gaythorne Ridsdale-Terry (1845–1913) ...",wiki,mpt,none,1
3,Shark Tale is doubtlessly going to be endlessl...,reviews,human,number,0
4,We develop a system for measurements of powe...,abstracts,human,upper_lower,0
...,...,...,...,...,...
227995,1 cup water\n1/2 cup (1 stick) unsalted butter...,recipes,human,perplexity_misspelling,0
227996,After years of aimless wanderings\nLeaving beh...,poetry,human,none,0
227997,"\n\n""Looper"" is a great movie. It's a smart, w...",reviews,gpt3,alternative_spelling,1
227998,"\n\nIn 1871, two young boys, Dick and Harry, r...",books,gpt3,upper_lower,1


In [30]:
df['labels'].value_counts()

labels
1    138000
0     90000
Name: count, dtype: int64

In [65]:
df.to_csv('raid_twitter_train.csv', index=False)